In [1]:
import Rust_Machine_Learning
import plotly.express as px
import pandas as pd

In [2]:
# Get the csv data
# df = pd.read_csv("../data/iris.data")
# df = pd.read_csv("../data/kmeans-dataset.csv")
# df = pd.read_csv("../data/winequality-red.csv")
df = pd.DataFrame(
    data={
        "Loves Popcorn": [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0],
        "Loves Soda": [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0],
        "Age": [7.0, 12.0, 18.0, 35.0, 38.0, 50.0, 83.0],
        "Loves Cool As Ice": [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0],
    }
)
df


,Loves Popcorn,Loves Soda,Age,Loves Cool As Ice
0,1.0,1.0,7.0,0.0
1,1.0,0.0,12.0,0.0
2,0.0,1.0,18.0,1.0
3,0.0,1.0,35.0,1.0
4,1.0,1.0,38.0,1.0
5,1.0,0.0,50.0,0.0
6,0.0,0.0,83.0,0.0


In [3]:
# Ensure categoriical data is one-hot encoded
# columns is list of ints
def one_hot_encode(df, columns):
    encoded = df.copy()
    for col in columns:
        # Get the unique values
        unique_vals = encoded.iloc[:, col].unique()

        # Now for each unique value add a col to df
        for val in unique_vals:
            col_encoded = encoded.iloc[:, col] == val
            col_encoded = [(1.0 if label == True else 0.0) for label in col_encoded]
            encoded["is_" + encoded.columns[col] + "_" + str(val)] = col_encoded
    encoded.drop(encoded.columns[columns], axis=1, inplace=True)
    return encoded


In [4]:
categories_to_encode = []
encoded = one_hot_encode(df, categories_to_encode)
encoded


,Loves Popcorn,Loves Soda,Age,Loves Cool As Ice
0,1.0,1.0,7.0,0.0
1,1.0,0.0,12.0,0.0
2,0.0,1.0,18.0,1.0
3,0.0,1.0,35.0,1.0
4,1.0,1.0,38.0,1.0
5,1.0,0.0,50.0,0.0
6,0.0,0.0,83.0,0.0


In [5]:
# Split the data into train and test
def split_train_test(df, percent_train, randomize = True):
  #Randomize the rows
  if randomize:
    df = df.sample(frac=1.0)

  #Create train and test data
  df_size = df.shape[0]
  train_size = df_size * percent_train

  train_data = df.iloc[:int(train_size),:]
  test_data = df.iloc[int(train_size):,:]
  return (train_data, test_data)

train_data, test_data = split_train_test(encoded, 1.0, False)

In [6]:
# Get categorical based on indices
def get_is_categorical(categorical_original, df_original, encoded):
  is_categorical = []

  all_categories = categorical_original
  if(df_original.shape[1] != encoded.shape[1]):
    all_categories.extend(range(df_original.shape[1], encoded.shape[1]))

  for i in range(encoded.shape[1]):
    is_categorical.append(True if i in all_categories else False)
  return is_categorical

In [7]:
# Now train the model
label_col = 3
train_data_features = train_data.copy()
train_data_features.drop(train_data.columns[label_col], axis=1, inplace=True)
train_data_features = train_data_features.values.tolist()
train_data_labels = train_data[train_data.columns[label_col]].values.tolist()

is_categorical = get_is_categorical([0, 1], df, encoded)
del is_categorical[label_col]

for i in range(1):
    trained_model = Rust_Machine_Learning.DecisionTree(
        train_data_features, is_categorical, train_data_labels
    )


{1: (0.21428571428571427, 0.0), 0: (0.40476190476190477, 0.0), 2: (0.34285714285714286, 44.0)}
